# Preprocessing Notebook

In [12]:
import pandas as pd
import geopandas as gpd
from ydata_profiling import ProfileReport

## First, air stations pre-processing

In [2]:
air_stations = pd.read_csv('data/air_quality_stations.csv')

In [6]:
air_stations.head(23)

,id,state,latitude,longitude
0,8,ACT,4.530214,-74.142217
1,18,ACT,4.940316,-73.970698
2,20,ACT,5.200000,-73.883000
3,4,ACT,4.596229,-74.194715
4,6,ACT,4.695702,-74.215583
5,17,ACT,4.862871,-74.056341
6,16,ACT,4.298055,-74.819167
7,23,ACT,4.594991,-74.204826
8,14,ACT,4.716525,-74.211712
9,22,ACT,4.809444,-74.102500


In [7]:
import plotly.express as px

# Crear el mapa interactivo con las estaciones
fig = px.scatter_map(
    air_stations,
    lat='latitude',
    lon='longitude',
    hover_name='id',  # Muestra el id al pasar el mouse
    hover_data={
        'latitude': ':.4f',  # Formato con 4 decimales
        'longitude': ':.4f',
        'state': True
    },
    zoom=8,  # Nivel de zoom inicial
    height=600,
    title='Estaciones de Medición de Calidad del Aire - Cundinamarca/Boyacá'
)

# Personalizar el diseño
fig.update_traces(marker=dict(size=12, color='red'))
fig.update_layout(
    map_style="open-street-map",  # Estilo de mapa OpenStreetMap
    margin={"r":0,"t":50,"l":0,"b":0}
)

fig.show()

## Emission permits

In [19]:
emission_permits = gpd.read_file('data/emission_permits_anom_2.json')
# Convertir GeoDataFrame a DataFrame normal
emission_permits['latitude'] = emission_permits.geometry.y
emission_permits['longitude'] = emission_permits.geometry.x
emission_permits = pd.DataFrame(emission_permits.drop(columns=['geometry']))

In [20]:
emission_permits.head()

,IDExpediente,Estado,Regional,Departamento,Municipio,Vereda,Class,TipoCombustible,TipoFuenteEmision,Cuenca,latitude,longitude
0,73640.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,MOSQUERA,CENTRO,None,Otros,Horno,Río Bogotá,4.703418,-74.226561
1,73788.0,Seguimiento y Control,Ubate,Cundinamarca,LENGUAZAQUE,Resguardo,None,Carbón,Caldera Horno,Río Suárez,5.318407,-73.704281
2,74314.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,MADRID,LA PUNTA,None,ACPM,Caldera Horno,Río Bogotá,4.800462,-74.210355
3,75972.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,MOSQUERA,Balsillas,None,Fuel Oil No.8,Planta de Asfalto,Río Bogotá,4.678797,-74.284112
4,78824.0,Seguimiento y Control,Sabana Occidente,Cundinamarca,FUNZA,El Hato,None,Carbón,Caldera Horno,Río Bogotá,4.699590,-74.193752


In [21]:
report = ProfileReport(emission_permits, title="Emission Permits Data Profile", explorative=True)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 12/12 [00:00<00:00, 4684.63it/s]



Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

### Perfilamiento:
- **IDExpediente**: Identificador único, aunque solo 92.3% de los IDs son diferentes, probablemente algunas compañías tienen múltiples ubicaciones
- **Estado**: La mayoría de las estaciones están en estado Sancionatorio (321). Hay 7 "En Trámite" y 216 en "Seguimiento y control".
- **Regional**: De la región donde se ubica el lugar. Puede ser útil para filtros regionales. 14 Distintos.
- **Departamento**: Puede ser útil para filtros departamentales. 3 Distintos
- **Municipio**: Puede ser útil para filtros municipales. 65 Distintos.
- **Vereda**: Muchos valores distintos (148) Omitiría este filtro
- **Class**: 41% de los campos son null. Solo no es null para los permisos de emisión en estado "Sancionatorio".
- **TipoCombustible**: Toma valor "(sin definir)" para permisos en estado "Sancionatorio" y "En Trámite"
- **TipoFuenteEmision**: Igual que el campo anterior
- **Cuenca**: Solo 77 datos no tienen una cuenca asociada
- **latitude** y **longitude**: Hay unos cuantos datos corridos en el mapa. Deben ser arreglados.

In [22]:
emission_permits.drop_duplicates(inplace=True)